In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
income_df = pd.read_csv("../data/income/2021_income.csv")

In [6]:
income_df

,POA_CODE_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size
0,2000,32,2800,941,625,2367,1.3,2225,2.1
1,2007,30,2500,772,500,2197,1.2,1805,2.1
2,2008,28,2600,860,525,2453,1.2,1746,1.9
3,2009,37,2800,1297,580,3035,1.1,2422,2.1
4,2010,36,2900,1479,550,3709,1.1,2297,1.7
...,...,...,...,...,...,...,...,...,...
2638,2899,50,1300,736,240,1630,0.8,1184,2.1
2639,6798,38,1300,1233,150,2659,1.0,2109,2.5
2640,6799,40,1972,741,231,1678,1.1,2519,3.8
2641,9494,0,0,0,0,0,0.0,0,0.0


In [3]:
merchant_detail = pd.read_parquet('../data/curated/merchant_detail.parquet')

In [4]:
merchant_detail

,merchant_name,tags,take_rate,total_income,total_transactions,mean_transaction_amount,monthly_profit
0,Amet Risus Inc.,"([furniture, home furnishings and equipment sh...",6.82,8657277.10,4251,2037.0,32801.0
1,Dignissim Maecenas Foundation,"[[opticians, oPtical goods, and eyeglasses], [...",6.64,8316735.67,58685,142.0,30680.0
2,Orci In Consequat Corporation,"([gift, card, novelty, and souvenir shops], [a...",6.61,8339994.52,75853,110.0,30626.0
3,Leo In Consulting,"[[watch, clock, and jewelry repair shops], [a]...",6.43,8443178.70,241336,35.0,30161.0
4,Etiam Bibendum Industries,"[(tent and awning shops), (a), (take rate: 6.31)]",6.31,8482176.66,6341,1338.0,29735.0
...,...,...,...,...,...,...,...
4021,Feugiat Nec Diam LLC,"([computers, computer peripheral equipment, an...",0.30,18710.78,703,27.0,3.0
4022,Proin Vel Inc.,"[[stationery, office supplies and printing and...",0.16,21967.78,31,709.0,2.0
4023,Hendrerit Donec Limited,"[(digital goods: books, movies, music), (e), (...",0.18,20034.94,590,34.0,2.0
4024,Turpis Incorporated,"[[gift, card, novelty, and souvenir shops], [e...",0.23,16159.06,156,104.0,2.0
